In [76]:
import numpy as np
import copy

def random():
    return (np.random.random() - 0.5) * 2

In [83]:
class Node():
    def __init__(self, num, layer, func):
        self.num = num
        self.func = func
        self.layer = layer
        self.connections_in_num = 0
        self.connections_out = []
        self.in_values = []
        self.done = False
        self.value = 0
    
    def eval(self, value, connection):
        # Checking if the connection is valid
        if connection.out_node != self:
            print ("{} not connected to Node {}".format(connection, self.num))
            return None
        
        self.in_values.append(value)
        
        # does not check if a connection submited something twice, so don't run the net agsin until it is done
        if len(self.in_values) != self.connections_in_num:
            self.done = False
            print("Node: {} value recieve: {}, not complete".format(self.num, value))
        else:
            self.value = self.func(sum(self.in_values))
            for connection in self.connections_out:
                connection.eval(self.value, self)
            self.done = True
            print("Node: {} value recieve: {}, complete".format(self.num, value))
    
    def reset(self):
        self.done = False
        self.in_values = []
        self.value = 0

In [84]:
class Connection():
    def __init__(self, in_node, out_node, w, inno):
        self.in_node = in_node
        self.out_node = out_node
        self.weight = w
        self.innovation_num = inno
        self.enabled = True
    
    def mutate_weight(self, epsilon = .1):
        if np.random.random() < epsilon:
            self.weight = random()
        else:
            self.weight += np.random.normal()
        
        if self.weight > 1:
            self.weight = 1
        elif self.weight < -1:
            self.weight = -1
            
    def eval(self, value, node):
        if self.in_node != node:
            print ("Node {} is not connected to connection {}".format(node.num, self))
            return None
        elif self.enabled:
            self.out_node.eval(value*self.weight, self)
        else:
            return False

In [85]:
class Net():
    def __init__(self, num_in, num_out):
        self.in_nodes = []
        self.out_nodes = []
        self.nodes = []
        
        self.connections = []
        
        self.functions = [function]
        
        self.next_node_num = 0
        self.next_connection_num = 0
        
        # in nodes
        for i in range(num_in):
            self.in_nodes.append(Node(self.next_node_num, 0, function))
            self.nodes.append(self.in_nodes[-1])
            self.next_node_num += 1
        
        self.bias_node = Node(self.next_node_num, 0, bias)
        self.nodes.append(self.bias_node)
        self.next_node_num += 1
        
        # out nodes
        for i in range(num_out):
            self.out_nodes.append(Node(self.next_node_num, 2, function))
            self.nodes.append(self.out_nodes[-1])
            self.next_node_num += 1
        
        # connecting all in nodes to out nodes
        for in_node in self.in_nodes:
            for out_node in self.out_nodes:
                self.connections.append(Connection(in_node, out_node, random(), self.next_connection_num))
                self.next_connection_num += 1
        
        #connecting bias node to out nodes
        for out_node in self.out_nodes:
            self.connections.append(Connection(self.bias_node, out_node, random(), self.next_connection_num))
            self.next_connection_num += 1
        
        # adding connections to nodes
        self.connect()
        
        #setting up input nodes to work
        for node in self.in_nodes:
            node.connections_in_num = 1
            
        self.bias_node.connections_in_num = 1
        
    def eval(self, in_values):
        
        out_values = []
        
        #inserting input values
        for i in range(len(self.in_nodes)):
            self.in_nodes[i].eval(in_values[i], Connection(None, self.in_nodes[i], 1, -1))
        
        self.bias_node.eval(0,  Connection(None, self.bias_node, 1, -1))
        
        #making sure each node is done
        for node in self.nodes:
            while not node.done:
                pass
            print("Node: {} done".format(node.num))
        
        #getting output values
        for node in self.out_nodes:
            out_values.append(node.value)
        
        self.reset()
        
        return out_values
    
    def reset(self):
        # Reset all nodes
        for node in self.nodes:
            node.reset()
    
    def connect(self):
        # Reset all connections of each node
        
        for node in self.nodes:
            node.connections_out = []
            node.connections_in_num = 0
        
        for connection in self.connections:
            if connection.enabled:
                connection.in_node.connections_out.append(connection)
                connection.out_node.connections_in_num += 1
            
    def add_node(innovation_history):
        
        connection = np.random.choice(self.connections)
        while connection.in_node == self.bias_node:
            connection = np.random.choice(self.connections)
        
        connection.enabled = False
        
        new_node = Node(self.next_node_num, 1, np.random.choice(self.functions))
        self.next_node_num += 1
        self.nodes.append(new_node)
        
        inno_num = new_innovation_num(innovation_history)
        self.connections.append(Connection(new_node, connection.out_node, connection.weight, inno_num))
        
        inno_num = new_innovation_num(innovation_history)
        self.connections.append(Connection(connection.in_node, new_node, 1, inno_num))
        
    def new_innovation_num(innovation_history, connection):
        # Checking if the innovation already exists, and gets the number
        for innovation in innovation_history:
            if innovation.matches(self, connection):
                return innovation.innovation_num
        
        # If not already an innovation add it to the history
        innovation_nums = []
        for connection in self.connections:
            innovation_nums.append(connection.num)
            
        innovation_history.append(Innvation(connection.in_node, connection.out_node, self.next_connection_num, innovation_nums))
        self.next_connection_num += 1
        return self.next_connection_num - 1
        

In [100]:
class Innovation():
    def __init__(self, in_node, out_node, innovation_num, innovation_nums):
        self.in_node = in_node
        self.out_node = out_node
        self.innovation_num = innovation_num
        self.innovation_nums = copy.deepcopy(innovation_nums)
    
    def matches(self, net, connection):
        if len(net.connections) == len(self.innovation_nums):
            if (connection.in_node.num == self.in_node.num) and (connection.out_node.num == self.out_node.num):
                for connection in net.connections:
                    if not (connection in self.innovation_nums):
                        return False
                    
                return True
            
        return False

In [ ]:
class Trainer():
    def __init__(self):
        self.global_next_innovation_num = 0
        self.global_innovation_history = []

In [86]:
def function(x):
    return x

def bias(x):
    return 1

In [95]:
net = Net(100, 100)
net.eval([1,2,3,4,5,6,7,8,9,10] *  10)

Node: 101 value recieve: -0.7271846012233236, not complete
Node: 102 value recieve: 0.5677126521989531, not complete
Node: 103 value recieve: 0.2387006284653963, not complete
Node: 104 value recieve: -0.05034938110365794, not complete
Node: 105 value recieve: -0.7670711087311692, not complete
Node: 106 value recieve: 0.8981584947156189, not complete
Node: 107 value recieve: 0.8321826286920684, not complete
Node: 108 value recieve: -0.5637957476774726, not complete
Node: 109 value recieve: -0.8621995279079648, not complete
Node: 110 value recieve: 0.5386133797713142, not complete
Node: 111 value recieve: 0.8952099452921847, not complete
Node: 112 value recieve: -0.9856021016030452, not complete
Node: 113 value recieve: 0.6075836810755775, not complete
Node: 114 value recieve: -0.8332436380942441, not complete
Node: 115 value recieve: 0.21572354628134316, not complete
Node: 116 value recieve: 0.7349709907081725, not complete
Node: 117 value recieve: -0.7579455512462094, not complete
Node

Node: 148 value recieve: 2.727443548309203, not complete
Node: 149 value recieve: 0.6825262064353721, not complete
Node: 150 value recieve: -1.2023909892845963, not complete
Node: 151 value recieve: -0.45512383110505095, not complete
Node: 152 value recieve: 4.439585783233536, not complete
Node: 153 value recieve: 4.215195777530796, not complete
Node: 154 value recieve: 2.932015497534487, not complete
Node: 155 value recieve: 1.044510981124842, not complete
Node: 156 value recieve: -2.837964087976589, not complete
Node: 157 value recieve: -2.4157559687450867, not complete
Node: 158 value recieve: -2.6838229586141704, not complete
Node: 159 value recieve: 3.7747067644902432, not complete
Node: 160 value recieve: 0.8401070504792829, not complete
Node: 161 value recieve: 0.7523059607281712, not complete
Node: 162 value recieve: -3.956676186146062, not complete
Node: 163 value recieve: -0.9683109364408138, not complete
Node: 164 value recieve: 3.0375018884245772, not complete
Node: 165 val

Node: 142 value recieve: -5.716033604429943, not complete
Node: 143 value recieve: -4.793586282661522, not complete
Node: 144 value recieve: 3.345227833420645, not complete
Node: 145 value recieve: -0.7555809748893729, not complete
Node: 146 value recieve: -0.1756882012587202, not complete
Node: 147 value recieve: -5.843042782646192, not complete
Node: 148 value recieve: -4.934120712285457, not complete
Node: 149 value recieve: -6.7016713052552, not complete
Node: 150 value recieve: -6.9301491426381885, not complete
Node: 151 value recieve: 1.7709959704681217, not complete
Node: 152 value recieve: 4.353982428112307, not complete
Node: 153 value recieve: 4.352829490044559, not complete
Node: 154 value recieve: -4.326459806880657, not complete
Node: 155 value recieve: 1.8430723029740432, not complete
Node: 156 value recieve: 1.6770601196293082, not complete
Node: 157 value recieve: 3.557236319854963, not complete
Node: 158 value recieve: 6.6799852216162705, not complete
Node: 159 value r

Node: 112 value recieve: -2.6003726411411305, not complete
Node: 113 value recieve: -3.148344871946674, not complete
Node: 114 value recieve: -0.48502307040201575, not complete
Node: 115 value recieve: -4.3442268021983805, not complete
Node: 116 value recieve: -1.9509606423841848, not complete
Node: 117 value recieve: 2.691603302761706, not complete
Node: 118 value recieve: -3.1234547129988695, not complete
Node: 119 value recieve: -1.8980928651544569, not complete
Node: 120 value recieve: 2.2394094603124164, not complete
Node: 121 value recieve: -5.483835700710836, not complete
Node: 122 value recieve: -2.618942001334309, not complete
Node: 123 value recieve: -4.746791469327334, not complete
Node: 124 value recieve: 6.279836302899136, not complete
Node: 125 value recieve: -5.0856146002383795, not complete
Node: 126 value recieve: -2.5069567065039777, not complete
Node: 127 value recieve: -6.4482728451002815, not complete
Node: 128 value recieve: -3.7906278124179646, not complete
Node:

[6.008732380839295,
 -14.263398670967499,
 52.35219135395811,
 -35.356234211158586,
 -27.25000324528893,
 -5.099489518150037,
 -1.6892732585205892,
 16.744569013542712,
 30.810725518859016,
 -59.90553252423431,
 -15.111431104919916,
 -9.194575992061356,
 15.458199808803762,
 -29.70646114466985,
 -39.24473927287587,
 -25.329807560016366,
 5.234671325509655,
 1.183382951815578,
 84.23088902937025,
 13.638329932268997,
 -32.84580661781461,
 -2.375890970580453,
 -14.670270372423914,
 -39.346401136847,
 48.64106287162988,
 7.2654392212521,
 -27.42589725696177,
 -32.25284035699763,
 36.543627164679236,
 20.136014167454505,
 9.640496268317504,
 -1.962554452121393,
 24.495062389073002,
 14.235770578901793,
 44.467506844982445,
 -25.75926886385639,
 -24.42060078319388,
 -44.387634166886144,
 -56.69599539422528,
 -23.71509081872543,
 66.2635455853464,
 -19.748414554277797,
 43.89435270991266,
 38.84403168046919,
 -29.67654981141929,
 -16.234911745687768,
 -45.04331476791621,
 19.80407510399207,


In [99]:
import copy

copy.copy(12)

12